In [2]:
from numpy.random import randint
def RandomPolynomial(degree, field_size):
    """
    degree: int, degree of polynomial
    field_size: int, size of finite field, must be a prime number in this case
    """
    k = GF(field_size)
    R.<x> = k[]
    list_random_coeff = randint(field_size, size = degree)
    sum_poly = 0
    for i,coeff in enumerate(list_random_coeff):
        sum_poly = sum_poly + coeff*x^i
    return sum_poly

def FindIrreduciblePolynomial(degree, field_size):
    """
    degree: int, degree of polynomial
    field_size: int, size of finite field, must be a prime number in this case
    """
    bool_irreducible = False
    count_test = 0
    while ((not bool_irreducible) and count_test<1000000):
        test_polynomial = RandomPolynomial(degree, field_size)
        bool_irreducible = test_polynomial.is_irreducible()
        count_test = count_test + 1
    return test_polynomial,count_test


In [3]:
# divison: relation bezou (algo naif)
def CoeffBezou(f,g):
    #return polynomials a, b so that 1 = a*f + b*g
    if(f.degree() == 0):
        return ((1+x-x) // f),0
    if(f == 0):
        raise Exception('f and g should have gcd 1')     
    if(g == 0):
        raise Exception('f and g should have gcd 1')  
    r = f % g
    h = f // g
    c,d = CoeffBezou(g,r)
    a = d
    b = c - d * h
    return a,b

# puissance: recursion div 2: division pour regner
def PowerPolynomial(f, n, pmod):
    #return f^n
    if(n == 0):
        return 1 + x - x
    elif(n == 1):
        return f % pmod
    elif(n % 2 == 1):
        half_pow = PowerPolynomial(f,n // 2, pmod)
        power_n = (half_pow * half_pow * f) % pmod
    else:
        half_pow = PowerPolynomial(f,n // 2, pmod)
        power_n = (half_pow * half_pow) % pmod
    return power_n

In [4]:
import numpy as np

def phi(f, q, pmod):
    #return f^q-f
    return (PowerPolynomial(f, q, pmod) - f) % pmod

def Check_Square(p, k):
    """Check if polynomial p has squared factors
    if yes, return the corresponding polynomial without squared factors
    """
    p_prime = p.derivative()
    p_mulFac = p.gcd(p_prime)
    has_square = (p_mulFac.degree() <= 0)
    p_noSq = p // p_mulFac
    return p_noSq, has_square
    
def Find_Factor_Berlekamp(p, k):
    """Find a factor of p using Berlekamp
    p : polynomial on field k without squared factor
    k : finite field
    """
    q = k.order()
    #Construct phi matrix
    coeff_list = []
    p_deg = p.degree()
    if p_deg <= 0:
        return "polynomial irreducible"
    for i in range(1, p_deg):
        f = x
        coeff_poly = phi(PowerPolynomial(f, i, p), q, p).list()
        for j in range(len(coeff_poly),p_deg):
            coeff_poly.append(0)
        coeff_list.append(coeff_poly)
    M = Matrix(coeff_list)
    ker = kernel(M)
    if(ker.dimension()==0):
        return "polynomial irreducible"
    element_non_const = ker.basis()[0]
    f = 0
    for i,coeff in enumerate(element_non_const):
        f = f + coeff*x^(i+1)
    for i,a in enumerate(k):
        pgcd = p.gcd(f-a)
        if not pgcd.degree() <= 0:
            return pgcd
    return "not found"

def Find_Factor_Berlekamp_Prob(p, k):
    """Find a factor of p using probabilistic Berlekamp
    p : polynomial on field k without squared factor
    k : finite field
    """
    q = k.order()
    p_deg = p.degree()
    if p_deg <= 0:
        return [p],True
    #Construct phi matrix
    coeff_list = []
    for i in range(0, p_deg):
        f = x
        coeff_poly = phi(PowerPolynomial(f, i, p), q, p).list()
        for j in range(len(coeff_poly),p_deg):
            coeff_poly.append(0)
        coeff_list.append(coeff_poly)
    M = Matrix(coeff_list)
    ker = kernel(M)
    ker_dim = ker.dimension()
    if(ker_dim==1):
        return [p],True
    #random element in kernel
    ker_base = ker.basis()
    element_non_const = ker_base[0]
    for i in range(ker_dim):
        element_non_const = element_non_const + k.random_element()*ker_base[i]
    f = 0
    for i,coeff in enumerate(element_non_const):
        f = f + coeff*x^i
    #test 0
    pgcd = p.gcd(f)
    if not pgcd.degree() <= 0:
        return [pgcd,p//pgcd],False   
    #phi(f) sont non-zero
    qq = int((q-1)/2)
    pm_one = PowerPolynomial(f, qq, p)
    pgcd = p.gcd(pm_one-1)
    if not pgcd.degree() <= 0:
        return [pgcd,p//pgcd],False
    return [p,1+x-x],False

def Factorize_Berlekamp_Prob(p, k):
    list_two_factors,irr = Find_Factor_Berlekamp_Prob(p, k)
    if irr:
        return list_two_factors
    else:
        return Factorize_Berlekamp_Prob(list_two_factors[0],k) + Factorize_Berlekamp_Prob(list_two_factors[1],k)

In [5]:
def Unity_Root(k):
    """Find a unity root of the field k
    """
    q = k.order()
    list_factor = factor(q-1)
    is_root = False
    i = 0
    while((not is_root) and i<q):
        i = i + 1
        is_root = True
        element_test = k.random_element()
        if element_test == 0:
            is_root = False
        for pi,_ in list_factor:
            if element_test^int(q/pi) == 1:
                is_root = False
    return element_test

def Mignotte_Schnorr_list(p, k, ksi, chi, list_pi):
    """Find a list of root using Mignotte-Schnorr
    p : splitting polynomial on field k, the polynomial to be factorized
    k : finite field 
    ksi : element of k, unity root of k of degree chi
    chi : integer, degree of ksi
    list_pi : list of prime integers, product of which equals chi
    """
    q = k.order()
    p_deg = p.degree()    
    rho = int((q-1)/chi)
    h0 = PowerPolynomial(x, rho, p)
    list_h = [h0]
    list_pi_popped = list_pi.pop()
    for pi in list_pi:
        hi = PowerPolynomial(list_h[-1], pi, p)
        list_h.append(hi)
    list_pi.append(list_pi_popped)
    F = [(p,0)]
    for i,hi in reversed(list(enumerate(list_h))):
        G = []
        pi = list_pi[i]
        for j in range(pi):
            for g,e in F:
                gj = g.gcd((hi % g) - ksi^int((e+j*chi)/pi))
                if gj.degree() >= 1:
                    G.append((gj,int((e+j*chi)/pi)))
        F = G
    return F

def Mignotte_Schnorr(p, k):
    """Find a list of root using Mignotte-Schnorr
    p : splitting polynomial on field k, the polynomial to be factorized
    k : finite field 
    """
    q = k.order()
    ksi = Unity_Root(k)
    pi = q-1
    pi_factor = factor(pi)
    list_pi = []
    for fac,rep in pi_factor:
        list_pi = list_pi + [fac]*rep
    F = Mignotte_Schnorr_list(p, k, ksi, pi, list_pi)
    list_root = [root for root,_ in F]
    return list_root

def Find_Root_CantorZassenhaus(p, k, ksi, chi):
    """Find factors of p using probabilistic Cantor-Zassenhaus
    p : polynomial whose factors have all degree 1, on field k, without squared factor
    k : finite field
    ksi : element of finite field, primitive root of unity of order ki
    chi : integer, order of ksi
    """
    q = k.order()
    p_deg = p.degree()
    if p_deg <= 1:
        return [p],True
    f = 0
    for i in range(p_deg):
        f = f + k.random_element()*x^i
    #test 0
    pgcd = p.gcd(f)
    if not pgcd.degree() <= 0:
        return [pgcd,p//pgcd],False
    rho = int((q-1)/chi)
    h = PowerPolynomial(f, rho, p)
    factor_list = [p]
    for i in range(1,chi-1):
        fi = factor_list[0].gcd(h-ksi^i)
        if not fi.degree()<=0:
            factor_list[0] = factor_list[0] // fi
            factor_list.append(fi)
    return factor_list,False

def Root_CantorZassenhaus(p, k, ksi, chi):
    list_factor,irr = Find_Root_CantorZassenhaus(p, k, ksi, chi)
    if irr:
        return list_factor
    else:
        list_return = []
        for factor in list_factor:
            list_return = list_return + Root_CantorZassenhaus(factor, k, ksi, chi)
        return list_return

In [6]:
def Factor_By_Degree(p, k):
    """Find factors by degree of p
    p : polynomial on field k, without squared factor
    k : finite field
    """
    q = k.order()
    p_deg = p.degree()
    list_factor = []
    p_rest = p
    phi_i = x
    for i in range(1,p_deg):
#TODO: Replace this with fast evaluation
        phi_i = PowerPolynomial(phi_i, q, p)
        e_i = p_rest.gcd(phi_i - x)
        list_factor.append(e_i)
        p_rest = p_rest//e_i
    return list_factor

def Find_Factor_CantorZassenhaus(p, k, d):
    """Find factors of p using probabilistic Cantor-Zassenhaus
    p : polynomial whose factors have all degree d, on field k, without squared factor
    k : finite field
    d : integer
    """
    q = k.order()
    p_deg = p.degree()
    if p_deg <= d:
        return [p],True
    f = 0
    for i in range(p_deg):
        f = f + k.random_element()*x^i
    #test 0
    pgcd = p.gcd(f)
    if not pgcd.degree() <= 0:
        return [pgcd,p//pgcd],False
#TODO: replace this with fast evaluation
    rho = int((q^d-1)/2)
    h = PowerPolynomial(f, rho, p)
    factor_list = [p]
    fi = factor_list[0].gcd(h-1)
    if not fi.degree()<=0:
        factor_list[0] = factor_list[0] // fi
        factor_list.append(fi)
    return factor_list,False
    
def Factor_Equidegree(e_i, k, deg_factor):
    """Find factors by degree of p
    e_i : polynomial whose factors have all degree deg_factor, on field k, without squared factor
    k : finite field
    deg_factor : integer, degree of irreducible factors of e_i 
    """
    list_factor,irr = Find_Factor_CantorZassenhaus(e_i, k, deg_factor)
    if irr:
        return list_factor
    else:
        list_return = []
        for factor in list_factor:
            list_return = list_return + Factor_Equidegree(factor, k, deg_factor)
        return list_return

def Factor_CantorZassenhaus(p, k):
    list_factor_equidegree = Factor_By_Degree(p, k)
    list_factor = []
    for i,e_i in enumerate(list_factor_equidegree):
        factor_e_i = Factor_Equidegree(e_i, k, i+1)
        list_factor = list_factor + factor_e_i
    return list_factor

In [7]:
def Multiple_Modulo(f,list_mod):
    """
    Calculate f mod m_i for multiple i
    f : polynomial
    list_mod : list of polynomials m_i
    """
    #Construction of tree
    r = len(list_mod)
    len_list_mod = len(list_mod)
    list_father = []
    i = 0
    j = 1
    while(j < r):
        list_mod.append(list_mod[i]*list_mod[j])
        list_father.append(r)
        list_father.append(r)
        r = r+1
        i = i+2
        j = j+2
    #print(list_mod, r,list_father)        
    #Modulo
    list_residual = []
    list_residual.append(f % list_mod[r-1])
    list_mod.pop()
    for pol_current,ind_father in zip(reversed(list_mod),reversed(list_father)):
        res = list_residual[r-1-ind_father] % pol_current
        list_residual.append(res)
    list_residual.reverse()
    result = list_residual[:len_list_mod]
    return result

def Chinese_Remainder(list_mod, list_remainder):
    """
    Find polynomial P so that P = b_i mod m_i for multiple i
    f : polynomial
    list_mod : list of polynomials m_i
    list_mod : list of polynomials b_i
    """
    #Calculate product of m_i: M(x)
    list_calc_M = list_mod[:]
    r = len(list_calc_M)
    i = 0
    j = 1
    while(j<r):
        list_calc_M.append(list_calc_M[i]*list_calc_M[j])
        r = r+1
        i = i+2
        j = j+2
    M = list_calc_M[-1]
    #Calculate M_i = M/m_i
    list_M_i = [M // m_i for m_i in list_mod]
    M_prime = M.derivative()
    M_prime_inv = CoeffBezou(M_prime,M)[0]
    list_M_prime_inv_mod_m_i = Multiple_Modulo(M_prime_inv, list_mod)
    list_m_i_prime = [m_i.derivative() for m_i in list_mod]
    list_M_i_inv = [(m_i_prime * M_prime_inv) % m_i 
                    for m_i_prime,M_prime_inv,m_i in zip(list_m_i_prime,list_M_prime_inv_mod_m_i,list_mod)]
    list_f_i = [(b_i * M_i_inv) % m_i 
                    for b_i,M_i_inv,m_i in zip(list_remainder,list_M_i_inv,list_mod)]
    list_sum_f_over_m = list_f_i[:]
    r = len(list_sum_f_over_m)
    i = 0
    j = 1
    while(j<r):
        list_sum_f_over_m.append(list_sum_f_over_m[i]*list_calc_M[j] + list_sum_f_over_m[j]*list_calc_M[i])
        r = r+1
        i = i+2
        j = j+2    
    return list_sum_f_over_m[-1]

def Fast_Iteration(poly_ring_f, f, list_poly, p_mod):
    """
    Calculate f(g_i) mod p_mod
    """
    quot_ring = poly_ring_f.quotient(p_mod, 'xbar')
    meta_ring = PolynomialRing(quot_ring, 'y')
    y = meta_ring.gen()
    list_y_minus = []
    for poly_eva in list_poly:
        list_y_minus.append(y - poly_eva)
    list_evaluation = Multiple_Modulo(f(y),list_y_minus)
    list_evaluation_x = []
    return list_evaluation

def Fast_Interpolation(list_point, list_value):
    return

In [36]:
k = GF(17)
R.<x> = k[]
f = x^4+3
list_mod = [x-1,x,x+1,x^2-3]
list_remainder = [4,3,4,12]
Chinese_Remainder(list_mod, list_remainder)

x^4 + 3

In [ ]:
k = GF(31)
R.<x> = k[]
p_mod = x^2+x+1
poly_ring_f = R
quot_ring = poly_ring_f.quotient(p_mod, 'xbar')
meta_ring = PolynomialRing(quot_ring, 'y')
y = meta_ring.gen()
Fast_Iteration(R, x^2+1, [x-1, x, x+1, x+2, x^2+1, x^2, x^3], x^5)